# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - Greedy Baseline</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Code modified from: https://www.kaggle.com/code/crodoc/1-187-898-greedy-baseline-improvement

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [1]:
from tqdm import tqdm


import pandas as pd
from ast import literal_eval
from dataclasses import dataclass
import random
from sympy.combinatorics import Permutation
from typing import Dict, List
import zipfile
import numpy as np
import sqlite3

In [2]:
database_file = '../solutions.db'
solution_method = 'greedy database update'

<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [3]:
with zipfile.ZipFile('../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col = 'id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

In [4]:
allowed_moves = {}

for idx, row in puzzle_info.iterrows():
    allowed_moves[idx] = eval(row['allowed_moves'])

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [5]:
def move_state(state, move, moves_pool):
    
    if '-' in move:
        move = move[1:]
        rev = True
    else:
        rev = False

    move = moves_pool[move]

    if rev:
        new_state = state[:]
        for i in range(len(move)):
            new_state[move[i]] = state[i]
        state = new_state
    else:
        state = [state[idx] for idx in move]
    
    return state

def solve_puzzle(puzzle_id, database_file):
    
    current_puzzle = puzzles.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    initial_state = current_puzzle['initial_state']
    solution_state = current_puzzle['solution_state']
    
    moves_pool = allowed_moves[puzzle_type]
    
    # Connect to the SQLite database
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()
    
    select_query = "SELECT moves FROM solutions WHERE id = ?"
            
    # Execute the query
    cursor.execute(select_query, (game_id,))
    moves = cursor.fetchone()[0].split('.')
    
    # moves = submission.loc[puzzle_id]['moves'].split('.')
    
    
    state = initial_state.split(';')
    
    state_list = []
    state_list.append(state)
    
    for move in moves:
        state = move_state(state, move, moves_pool)
        state_list.append(state)
    
    state_to_idx = {}
    for idx in range(len(state_list)):
        state_to_idx[';'.join(state_list[idx])] = idx
    
    res = []
    idx = 0
    
    # ignore last state
    for curr_idx in tqdm(list(range(len(state_list)-1))):
        
        if curr_idx != idx:
            continue
        
        state = state_list[idx]
        
        new_idx = -1
        new_move = ''
        
        for move in moves_pool:
            for reversed_move in ['', '-']:
                move = reversed_move + move
                
                new_state = ';'.join(move_state(state, move, moves_pool))
                
                if new_state in state_to_idx:
                    tmp_idx = state_to_idx[new_state]

                    if tmp_idx > new_idx:
                        new_idx = tmp_idx
                        new_move = move
        
        idx = new_idx
        res.append(new_move)
            
    print('PUZZLE_ID:', puzzle_id)
    print('MOVES BEFORE:', len(state_list)-1)
    print('MOVES AFTER:', len(res))
    print()
    
    return '.'.join(res)

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [13]:
database_file = '../solutions.db'
solution_method = 'greedy baseline'

# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()
for game_id in puzzles.index:
    
    if game_id not in range(281, 282):
        continue
    
    solution = solve_puzzle(game_id, database_file)
    move_count = len(solution.split('.'))
    
    select_query = "SELECT count, solution_method FROM solutions WHERE id = ?"
        
    # Execute the query
    cursor.execute(select_query, (game_id,))
    response = cursor.fetchone()
    
    best_move_count = response[0]
    best_solution_method = response[1]
    
    if best_move_count > move_count:
        # Insert the moves into the database
        insert_query = "INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)"
        cursor.execute(insert_query, (game_id, solution, move_count, f'{solution_method}: {best_solution_method}'))
        conn.commit()

# Commit the changes and close the connection
conn.commit()
conn.close()

100%|██████████| 84619/84619 [1:50:13<00:00, 12.79it/s]  


PUZZLE_ID: 281
MOVES BEFORE: 84619
MOVES AFTER: 83567
